In [2]:
import pandas as pd
import os
import numpy as np
import ast
import pysam
import collections
from tqdm import tqdm
from Bio.Seq import Seq

In [3]:
def quickFilter(df):
    REASONING=[]
    sidePieces=[]

    df2 = df.copy()
    
    for row in df2.index:
        
        flags=[]
        flag=0

        if str(df2.at[row, 'TE_Designation']) == 'SINE/Alu':
            
            ############################################

            if int(df2.at[row, 'Sequence_Length'])>500:
                flag+=1
                flags.append("Alu_>500Len")
            else:
                pass
            
            
            ############################################
                    
            if flag==0:
                REASONING.append('Good_Row')
            else:
                REASONING.append(flags)
            
        elif str(df2.at[row, 'TE_Designation']) == 'LINE/L1':
        
            ############################################

            if int(df2.at[row, 'Sequence_Length'])>10000:
                flag+=1
                flags.append("LINE_>10kLen")
            else:
                pass
            
            if flag==0:
                REASONING.append('Good_Row')
            else:
                REASONING.append(flags)
        
        elif str(df2.at[row, 'TE_Designation']) == 'Retroposon/SVA':
        
            ############################################

            if int(df2.at[row, 'Sequence_Length'])>10000:
                flag+=1
                flags.append("SVA_>10kLen")
            else:
                pass
            
            ############################################

            if flag==0:
                REASONING.append('Good_Row')
            else:
                REASONING.append(flags)
        
        
        else:
            REASONING.append('Good_Row')
            
            
    df2['FILTER_RESULTS']=REASONING
    return(df2)

In [4]:
directory = '/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/'
elementList = ['LINE/L1','snRNA','srpRNA','Retroposon/SVA','SINE/Alu']
#completeRunsSpecies=['hs1','PanTro']
#completeRunsSpecies=['PanPan']

speciesLSDirectory={}
for speciesDataframe in os.listdir(directory):
    
    if '6nt.csv' in speciesDataframe:
        
        #Current Species Name EX: GorGor
        speciesName = str(speciesDataframe.split("_")[0])
        
        #Add species to dictionary
        speciesLSDirectory[speciesName]={}
        
        #Read in the INSERTION DATAFRAME
        preworkingDF = pd.read_csv(directory+speciesDataframe).set_index("ID")
        workingDF = preworkingDF[(preworkingDF['TE_Percentage']>=0.2) & (preworkingDF['A-Tail_Length']>0) & (preworkingDF['TE_Designation'].isin(elementList))].copy()
        filteredDF = quickFilter(workingDF)
        filteredDF2 = filteredDF[filteredDF['FILTER_RESULTS']=='Good_Row'].copy()
        
        #Build Dictionary
        for row in filteredDF2.index:
            speciesLSDirectory[speciesName][row]={}
            
        #clean memory
        del preworkingDF, workingDF, filteredDF
        
        #Species File is the blat file
        for speciesFile in os.listdir('/project/mkonkel/tangeno/TEleaves/LS_Primates/blat/'):
            
            if '.fasta' in str(speciesFile) or 'txt.tmp' in str(speciesFile):
                continue
            else:
                
                if speciesName == str(speciesFile.split("-")[0]):
                    
                    #Which species it blatted against
                    speciesTwo = str('-'.join(speciesFile.split("_blat")[0].split("-")[1:]))
                
                    print(speciesName, speciesTwo, speciesFile)
                    #Read in blat file
                    f = open('/project/mkonkel/tangeno/TEleaves/LS_Primates/blat/'+str(speciesFile),'r')
                    lines = f.readlines()[5:]
                    blatDF = pd.DataFrame(data=[x.split("\t") for x in lines])
                    blatDF[0]=blatDF[0].astype(int)
                    blatDFF = blatDF[blatDF[9].isin(speciesLSDirectory[speciesName].keys())].copy()
                    del blatDF
                    f.close()

                    if speciesName == speciesTwo:
                        
                        goodRows=[]
                        for row in tqdm(blatDFF.index):
                            size = int(filteredDF2.at[blatDFF.at[row,9], 'Sequence_Length'])

                            contig = str(blatDFF.at[row,9].split(":")[0])
                            start = int(blatDFF.at[row,9].split(":")[1].split("-")[0])-500

                            blatContig = str(blatDFF.at[row,13])
                            blatStart = int(blatDFF.at[row,15])

                            if contig == blatContig and abs(start-blatStart)<=100:
                                continue
                            elif abs(size-int(blatDFF.at[row,7]))< (.3*size) and abs(start-blatStart)>100:
                                goodRows.append(row)
                            else:
                                continue
                    else:
                        goodRows=[]
                        for row in tqdm(blatDFF.index):
                            size = int(filteredDF2.at[blatDFF.at[row,9], 'Sequence_Length'])

                            contig = str(blatDFF.at[row,9].split(":")[0])
                            start = int(blatDFF.at[row,9].split(":")[1].split("-")[0])-500

                            blatContig = str(blatDFF.at[row,13])
                            blatStart = int(blatDFF.at[row,15])

                            if abs(size-int(blatDFF.at[row,7]))< (.3*size) and abs(start-blatStart)>100:
                                goodRows.append(row)
                            else:
                                continue

                    blatDF2 = blatDFF.loc[goodRows].copy()
                    del blatDFF

                    for row in blatDF2.index:
                        if speciesTwo in speciesLSDirectory[speciesName][str(blatDF2.at[row,9])].keys():
                            speciesLSDirectory[speciesName][str(blatDF2.at[row,9])][speciesTwo].append(str(blatDF2.at[row,8])+"_"+str(blatDF2.at[row,13])+":"+str(blatDF2.at[row,15])+"-"+str(blatDF2.at[row,16]))
                        else:
                            speciesLSDirectory[speciesName][str(blatDF2.at[row,9])][speciesTwo]=[]
                            speciesLSDirectory[speciesName][str(blatDF2.at[row,9])][speciesTwo].append(str(blatDF2.at[row,8])+"_"+str(blatDF2.at[row,13])+":"+str(blatDF2.at[row,15])+"-"+str(blatDF2.at[row,16]))
                else:
                    continue

    else:
        continue

GorGor GorGor-mat GorGor-GorGor-mat_blat.txt


100%|██████████| 378968/378968 [00:07<00:00, 50938.96it/s]


GorGor GorGor-pat GorGor-GorGor-pat_blat.txt


100%|██████████| 373193/373193 [00:07<00:00, 52067.56it/s]


GorGor PanPan-mat GorGor-PanPan-mat_blat.txt


...

100%|██████████| 636460/636460 [00:12<00:00, 51738.08it/s]


hs1 SymSyn-pri hs1-SymSyn-pri_blat.txt


100%|██████████| 681091/681091 [00:13<00:00, 51607.15it/s]


hs1 hs1.txt hs1-hs1.txt


100%|██████████| 724267/724267 [00:14<00:00, 50917.85it/s]


In [5]:
orang1List =[]
for loci in tqdm(speciesLSDirectory['PonAbe'].keys()):
    for species in speciesLSDirectory['PonAbe'][loci].keys():
        orang1List.append(['PonAbe',loci, species, speciesLSDirectory['PonAbe'][loci][species]])
PonAbeDF = pd.DataFrame(data=orang1List, columns=['Focus_Species','Loci','Query_Species','Matches'])

100%|██████████| 4653/4653 [00:00<00:00, 957845.23it/s]


In [6]:
panpan1List =[]
for loci in tqdm(speciesLSDirectory['PanPan'].keys()):
    for species in speciesLSDirectory['PanPan'][loci].keys():
        panpan1List.append(['PanPan',loci, species, speciesLSDirectory['PanPan'][loci][species]])
PanPanDF = pd.DataFrame(data=panpan1List, columns=['Focus_Species','Loci','Query_Species','Matches'])

100%|██████████| 4712/4712 [00:00<00:00, 793048.45it/s]


In [7]:
orang2List =[]
for loci in tqdm(speciesLSDirectory['PonPyg'].keys()):
    for species in speciesLSDirectory['PonPyg'][loci].keys():
        orang2List.append(['PonPyg',loci, species, speciesLSDirectory['PonPyg'][loci][species]])
PonPygDF = pd.DataFrame(data=orang2List, columns=['Focus_Species','Loci','Query_Species','Matches'])

100%|██████████| 5145/5145 [00:00<00:00, 914626.35it/s]


In [8]:
humanList =[]
for loci in tqdm(speciesLSDirectory['hs1'].keys()):
    for species in speciesLSDirectory['hs1'][loci].keys():
        humanList.append(['hs1',loci, species, speciesLSDirectory['hs1'][loci][species]])
hs1DF = pd.DataFrame(data=humanList, columns=['Focus_Species','Loci','Query_Species','Matches'])

100%|██████████| 17492/17492 [00:00<00:00, 775358.69it/s]


In [9]:
gorillaList =[]
for loci in tqdm(speciesLSDirectory['GorGor'].keys()):
    for species in speciesLSDirectory['GorGor'][loci].keys():
        gorillaList.append(['GorGor',loci, species, speciesLSDirectory['GorGor'][loci][species]])
gorillaDF = pd.DataFrame(data=gorillaList, columns=['Focus_Species','Loci','Query_Species','Matches'])

100%|██████████| 13214/13214 [00:00<00:00, 1171868.76it/s]


In [10]:
symsynList =[]
for loci in tqdm(speciesLSDirectory['SymSyn'].keys()):
    for species in speciesLSDirectory['SymSyn'][loci].keys():
        symsynList.append(['SymSyn',loci, species, speciesLSDirectory['SymSyn'][loci][species]])
SymSynDF = pd.DataFrame(data=symsynList, columns=['Focus_Species','Loci','Query_Species','Matches'])

100%|██████████| 112017/112017 [00:00<00:00, 230848.09it/s]


In [11]:
panTroList =[]
for loci in tqdm(speciesLSDirectory['PanTro'].keys()):
    for species in speciesLSDirectory['PanTro'][loci].keys():
        panTroList.append(['PanTro',loci, species, speciesLSDirectory['PanTro'][loci][species]])
PanTroDF = pd.DataFrame(data=panTroList, columns=['Focus_Species','Loci','Query_Species','Matches'])

100%|██████████| 4625/4625 [00:00<00:00, 839078.51it/s]


In [12]:
#gorillaDF.to_csv("/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/GorGor_Matches_06-12-2024.csv")
#PonAbeDF.to_csv("/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/PonAbe_Matches_06-12-2024.csv")
#PonPygDF.to_csv("/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/PonPyg_Matches_06-12-2024.csv")
#hs1DF.to_csv("/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/hs1_Matches_06-12-2024.csv")
#SymSynDF.to_csv("/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/SymSyn_Matches_06-12-2024.csv")
#PanTroDF.to_csv("/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/PanTro_Matches_06-12-2024.csv")
#PanPanDF.to_csv("/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/PanPan_Matches_06-12-2024.csv")

In [13]:
gorillaDF

,Focus_Species,Loci,Query_Species,Matches
0,GorGor,CM055446.2:2790607-2795782,GorGor-mat,"[+_CM068942.1:2686964-2693388, +_CM068942.1:35..."
1,GorGor,CM055446.2:2790607-2795782,GorGor-pat,"[+_CM068948.1:7001966-7008134, +_CM068948.1:71..."
2,GorGor,CM055446.2:3569409-3570230,GorGor-mat,"[+_CM068942.1:5275988-5277844, +_CM068942.1:80..."
3,GorGor,CM055446.2:3569409-3570230,GorGor-pat,"[+_CM068948.1:3429423-3431260, +_CM068948.1:17..."
4,GorGor,CM055446.2:3569409-3570230,PanPan-mat,"[+_CM054490.2:139563004-139564933, -_CM054492...."
...,...,...,...,...
19863,GorGor,CM068951.1:47457211-47457321,hs1.txt,[+_NC_060946.1:49918908-49920020]
19864,GorGor,CM068951.1:47794062-47794371,GorGor-pat,[+_CM068949.1:47386348-47387656]
19865,GorGor,CM068951.1:48322803-48323446,GorGor-pat,[+_CM068949.1:47917838-47919482]
19866,GorGor,CM068951.1:48440016-48442551,GorGor-pat,[+_CM068949.1:48039846-48043280]


## DO SOME MANUAL CHECKS

In [59]:
checkList = ['CM054633.2:184574742-184575061',
'CM054640.2:7961999-7962321',
'CM054640.2:7996341-7996663',
'CM054637.2:136063218-136063533',
'CM054635.2:167931986-167932290',
'CM054633.2:95818663-95818867',
'CM054633.2:84724318-84724475',
'CM054634.2:117623306-117623642']

checkList2 = [
'CM054633.2:95818663-95818867',
'CM054633.2:84724318-84724475',
'CM054634.2:117623306-117623642']

In [74]:
f = open('/project/mkonkel/tangeno/TEleaves/LS_Primates/blat/PonPyg_PonPyg_blat.txt','r')
lines = f.readlines()[5:]
blatDF = pd.DataFrame(data=[x.split("\t") for x in lines])
blatDF[0]=blatDF[0].astype(int)
blatDFF = blatDF[blatDF[9].isin(speciesLSDirectory['PonPyg'].keys())].copy()

In [75]:
for x in blatDFF[blatDFF[9].isin(checkList2)].index:
    print([y for y in blatDFF.loc[x]])

[1002, '0', '0', '0', '0', '0', '1', '156', '+', 'CM054633.2:84724318-84724475', '1002', '0', '1002', 'CM054633.2', '203829688', '84723817', '84724975', '2', '501,501,', '0,501,', '84723817,84724474,\n']
[1002, '0', '0', '0', '0', '0', '1', '203', '+', 'CM054633.2:95818663-95818867', '1002', '0', '1002', 'CM054633.2', '203829688', '95818162', '95819367', '2', '501,501,', '0,501,', '95818162,95818866,\n']
[1002, '0', '0', '0', '0', '0', '1', '335', '+', 'CM054634.2:117623306-117623642', '1002', '0', '1002', 'CM054634.2', '186348328', '117622805', '117624142', '2', '498,504,', '0,498,', '117622805,117623638,\n']


In [64]:
speciesLSDirectory['PonPyg']['CM054633.2:84724318-84724475']

{}

In [27]:
df = pd.read_csv('/project/mkonkel/tangeno/TEleaves/LS_Primates/dataframes/PonPyg_sv_wRM_wTails_6nt.csv').set_index("ID")
workingDF = df[(df['TE_Percentage']>=0.75) & (df['A-Tail_Length']>0) & (df['TE_Designation'].isin(elementList))].copy()        

In [67]:
filteredDF = quickFilter(workingDF.loc[checkList])

In [68]:
filteredDF

,Sequence,Sequence_Length,TE_Percentage,TE_Hits,TE_Designation,TE_Proportion,A-Tail_Length,Double_Tail_Warning,Pos_TSD,Pos_TSD_Length,FILTER_RESULTS
ID,,,,,,,,,,,
CM054633.2:184574742-184575061,ccgggcgtggtagctcatgcctgtaatcccagtactttgggaggcc...,319,0.956113,['357 4.2 0.0 0.3 CM054633.2:184574742-1845750...,SINE/Alu,{'SINE/Alu': 0.9561128526645768},29,NaN,GGAGAAGGGA,10,Good_Row
CM054640.2:7961999-7962321,ttttttttttttttttttttttttttttgagacagagtctggctct...,322,0.956522,['327 6.4 0.0 0.0 CM054640.2:7961999-7962321 1...,SINE/Alu,{'SINE/Alu': 0.9565217391304348},28,NaN,NaN,0,Good_Row
CM054640.2:7996341-7996663,ggctggtcttgaactcttttttttttttttttttttttttttttga...,322,0.947205,['319 6.8 0.0 0.4 CM054640.2:7996341-7996663 1...,SINE/Alu,{'SINE/Alu': 0.9472049689440993},28,NaN,GGCTGGTCTTGAACTC,16,Good_Row
CM054637.2:136063218-136063533,AAAAATGggccgggcgtggtggctcacgcctgtaatcctagcactt...,315,0.942857,['340 5.0 0.7 0.0 CM054637.2:136063218-1360635...,SINE/Alu,{'SINE/Alu': 0.9428571428571428},19,NaN,GAGAGTATGT,10,Good_Row
CM054635.2:167931986-167932290,AGAAAATACTCTCTgtagctcacgcctgtaatcccagcactttggg...,304,0.937500,['315 8.0 0.0 0.7 CM054635.2:167931986-1679322...,SINE/Alu,{'SINE/Alu': 0.9375},9,NaN,GAAAAGAAA,9,Good_Row
CM054633.2:95818663-95818867,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaattaaaaaagttagtt...,204,0.975490,['24 0.0 0.0 0.0 CM054633.2:95818663-95818867 ...,SINE/Alu,"{'Simple_repeat': 0.12254901960784313, 'SINE/A...",11,NaN,NaN,0,Good_Row
CM054633.2:84724318-84724475,tgtagtcccagctactcgggaggctgaggcagagaattgcttgaac...,157,0.904459,['144 9.8 0.0 0.0 CM054633.2:84724318-84724475...,SINE/Alu,{'SINE/Alu': 0.9044585987261147},21,NaN,GAAATGGGGCCAG,13,Good_Row
CM054634.2:117623306-117623642,aaaaaaaaaaaaaaaaaaaaaaaaaagaaaaaaaaggccgggcgcg...,336,0.988095,['30 2.9 0.0 0.0 CM054634.2:117623306-11762364...,SINE/Alu,"{'Simple_repeat': 0.10119047619047619, 'SINE/A...",13,NaN,GAAAA,5,Good_Row


In [73]:
speciesLSDirectory['PonPyg']['CM054637.2:136063218-136063533']

{}

In [69]:
tempDF = PonPygDF[PonPygDF['Loci'].isin(checkList)].copy()

In [ ]:
## RUN THE CODE BELOW ON PALMETTO SCRIPT

In [50]:
genomeDict ={
 'GorGor':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_029281585.2_NHGRI_mGorGor1-v2.0_pri_genomic.fna',
 'PanPan':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_029289425.2_NHGRI_mPanPan1-v2.0_pri_genomic.fna',
 'PanTro':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028858775.2_NHGRI_mPanTro3-v2.0_pri_genomic.fna',
 'PonAbe':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028885655.2_NHGRI_mPonAbe1-v2.0_pri_genomic.fna',
 'PonPyg':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028885625.2_NHGRI_mPonPyg2-v2.0_pri_genomic.fna',
 'SymSyn':'/project/mkonkel/tangeno/TEleaves/genomes/2024_Genomes/GCA_028878055.2_NHGRI_mSymSyn1-v2.0_pri_genomic.fna',
 'hs1':'/home/loftus/genomes/chm13v2/chm13v2.fasta',   
}

In [51]:
seqSims=[]
for row in tempDF.index:
    mykmerSize=14
    similarities=[]
    
    contig = str(tempDF.at[row,'Loci'].split(":")[0])
    
    start = str(int(tempDF.at[row,'Loci'].split(":")[1].split("-")[0])-500)
    end = str(int(tempDF.at[row,'Loci'].split(":")[1].split("-")[1])+500)
    
    sequence = ''.join(pysam.faidx(genomeDict['PonPyg'], str(contig)+":"+str(start)+"-"+str(end)).split()[1:])
    
    for match in ast.literal_eval(str(tempDF.at[row,'Matches'])):
        
        querySpecies = str(tempDF.at[row,'Query_Species'])
        
        if querySpecies == 'hs1':
            coordinate = str('_'.join(match.split("_")[1:]))
        else:
            coordinate = str(match.split("_")[1])
        
        if str(match.split("_")[0]) == '+':
            
            sequence2 = ''.join(pysam.faidx(genomeDict[querySpecies], coordinate).split()[1:])
            
        else:
            presequence = Seq(''.join(pysam.faidx(genomeDict[querySpecies], coordinate).split()[1:]))
            sequence2 = str(presequence.reverse_complement())

            
        kmers=[]
        insertionkmers=[]
        i=0
        while i < (len(sequence)-(mykmerSize-1)):
            kmers.append(str(sequence[i:i+mykmerSize]).upper())
            insertionkmers.append(str(sequence2[i:i+mykmerSize]).upper())
            i+=1

        uniqueKmers = set(kmers+insertionkmers)
        tempDF2 = pd.DataFrame(0, index=['Reference','MEI'], columns=[x for x in uniqueKmers])

        for kmer in kmers:
            tempDF2.at['Reference',kmer]+=1
        for kmer2 in insertionkmers:
            tempDF2.at['MEI',kmer2]+=1

        tempDF2.loc['Diff'] = tempDF2.loc['Reference'] - tempDF2.loc['MEI']
        tempDF2.loc['Sum'] = tempDF2.loc['Reference'] + tempDF2.loc['MEI']
        similarities.append(sum(abs(tempDF2.loc['Diff']))/sum(abs(tempDF2.loc['Sum'])))
        
    seqSims.append(similarities)
tempDF['Hit_Similarity']=seqSims

In [78]:
tempDF

,Focus_Species,Loci,Query_Species,Matches
232,PonPyg,CM054635.2:167931986-167932290,GorGor,[-_CM055451.2:191687611-191688871]
233,PonPyg,CM054635.2:167931986-167932290,PonAbe,[-_CM054684.2:169382614-169383873]
234,PonPyg,CM054635.2:167931986-167932290,PonPyg,[-_CM054635.2:168109275-168110532]
235,PonPyg,CM054635.2:167931986-167932290,hs1,[-_NC_060930.1:168741710-168742976]


In [83]:
167932290+500

167932790

In [84]:
print(pysam.faidx(genomeDict['PonPyg'], 'CM054635.2:167931486-167932790'))
for row in tempDF.index:
    for match in ast.literal_eval(str(tempDF.at[row,'Matches'])):
        #print(match)
        querySpecies = str(tempDF.at[row,'Query_Species'])
        
        if querySpecies == 'hs1':
            coordinate = str('_'.join(match.split("_")[1:]))
        else:
            coordinate = str(match.split("_")[1])
        
        if str(match.split("_")[0]) == '+':
            
            sequence2 = ''.join(pysam.faidx(genomeDict[querySpecies], coordinate).split()[1:])
            print(">"+str(querySpecies)+str(match))
            print(sequence2)
        else:
            
            presequence = Seq(''.join(pysam.faidx(genomeDict[querySpecies], coordinate).split()[1:]))
            sequence2 = str(presequence.reverse_complement())
            print(">"+str(querySpecies)+str(match))
            print(sequence2)

>CM054635.2:167931486-167932790
CAAGTCCCTCACTGAATTTGAAATATCAGATAAACAACAGTAATTTTTGGTATAAAAGTATGTTCCATGcaatatttgaa
atatacttatgttaaaaattattcatcatttggccgggtgtagtggctcacgcttgtaatcccagcaccctaggaggccg
aagcaggcagatcatctgaagtcaggagtctgagagcagcctggaaaacatggtgaaaccctgtctgtactaaaaataca
aaaattagccagacgtggtgacgcatgcctgtaattccagctactcaggaggctgaggcaggagaatcgcttgaactggg
gaggtggaggttgcagtgagccgagatcgtgccattgcactccagcctgggcaacagagtgagactctgtctcaaataat
aataataataataataaattattcatcatttatctaaaattcaaactTCACTGGCCATTTTGTATTATACCTGGCAACCC
TAGTTATAAAGCCAGATGATTAGAAAATACTCtctgtagctcacgcctgtaatcccagcactttgggaggcattggcggg
cggatcaggaggtagagagatcaagaccagcctggctaacacaggatgaaaccccgtctctactaaaaatacaaaacatt
agccgggcgtggtggcaggtgcctgtagtcccagctactcgggaggctgaggcaggagaatggcgtgaacccgggaggtg
gagcttgcagtgagctgagatcttgccactgcactccagcctgggcgacagagtgagactctgtctcaaaaaaaaagaaa
agaaaagaaaatactctcTATAAATCTCATGaatattttatagattatttGGATAGAACATGTTTCTAAACAGATTCAAT
GTAACATACACTGAAAAACTGCATTCTGCAGAAAAGTTCTACCCCAATGGCTCTAAAGAAATCGTGTTTCCCGACGG